In [1]:
!pip install aiohttp
!pip install aiofiles

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_latest_p37/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_latest_p37/bin/python -m pip install --upgrade pip' command.


In [ ]:
import boto3
import os
import sagemaker
import numpy as np
import urllib
import cv2
import csv
import ssl
import asyncio
import aiohttp
import aiofiles
import aiofiles.os
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
import asyncio
s3 = boto3.resource('s3')
csv_bucket = s3.Bucket('sagemaker-hotels50k-train') # example: energy_market_procesing
csv_obj = csv_bucket.Object(key='train/data-250k.csv') # example: market/zone1/data.csv

def return_response(url):
    return urllib.request.urlopen(url).read()


s3_client = boto3.client('s3')
data_bucket = 'hotel50k-data'
prefix = 'train'
# key = 'feature-vectors'


# get the object
response = csv_obj.get()
# read the contents of the file
lines = response['Body'].read()
lines = lines.decode('utf-8')
lines = lines.split()
# now iterate over those lines
iterator = 0
labels = []
pixelArray = []

#To ignore SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
counter = 0;
total_rows = len(list(csv.reader(lines)))-1


async def fetch(session, row):
    async with session.get(str(row[2])) as resp:
        if resp.status == 200:
            filename = str(row[0])+'.jpg'
            f = await aiofiles.open(filename, mode='wb')
            await f.write(await resp.read()) 
            object_name = os.path.join('hotel_classes', str(row[1]), filename)
            result = s3_client.upload_file(filename, 'hotel50k-data', object_name)
            await f.close()
            await aiofiles.os.remove(filename)
#             counter+=1;


async def getData():
    global counter
    async with aiohttp.ClientSession() as session:
        csv_reader = csv.reader(lines)
        next(csv_reader)
#         tasks = [fetch(session, row) for row in csv_reader]
        tasks = []
        for row in csv_reader:
            tasks.append(fetch(session,row))
            counter+=1;
            if counter % 500 == 0:
                exceptions = await asyncio.gather(*tasks, return_exceptions=True)
                print(exceptions)
                tasks = []
                clear_output(wait=True)
                display(str(round(counter/total_rows, 2))+'%')
        session.close()
        
await getData()


print('Done')


'0.85%'

In [17]:
!ls

ConvertImages.ipynb
Image-classification-fulltraining-elastic-inference.ipynb
Image-classification-fulltraining-highlevel.ipynb
Image-classification-fulltraining.ipynb
Image-classification-incremental-training-highlevel.ipynb
Image-classification-lst-format-highlevel.ipynb
Image-classification-lst-format.ipynb
Image-classification-transfer-learning-highlevel.ipynb
Image-classification-transfer-learning.ipynb
LoadImages.ipynb
